In [1]:
import pandas as pd
import glob
import re

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
psm_files = glob.glob("MSFragger_Output/*_psm.tsv")
psm_files

['MSFragger_Output\\acidorvorans_psm.tsv',
 'MSFragger_Output\\aurantiaca_psm.tsv',
 'MSFragger_Output\\baltica_psm.tsv',
 'MSFragger_Output\\comes_psm.tsv',
 'MSFragger_Output\\cryptum_psm.tsv',
 'MSFragger_Output\\denitrifans_psm.tsv',
 'MSFragger_Output\\gilvus_psm.tsv',
 'MSFragger_Output\\gnavus_psm.tsv',
 'MSFragger_Output\\griseorubens_psm.tsv',
 'MSFragger_Output\\indologenes_psm.tsv',
 'MSFragger_Output\\ljungdahlii_psm.tsv',
 'MSFragger_Output\\necator_psm.tsv',
 'MSFragger_Output\\palustris_psm.tsv',
 'MSFragger_Output\\smegmatis_psm.tsv',
 'MSFragger_Output\\subtilis_psm.tsv',
 'MSFragger_Output\\thermosulf_psm.tsv',
 'MSFragger_Output\\tumefaciens_psm.tsv']

In [4]:
run_files = glob.glob("MSFragger_Output/*_run.tsv")
run_files

['MSFragger_Output\\acidovorans_01_run.tsv',
 'MSFragger_Output\\acidovorans_02_run.tsv',
 'MSFragger_Output\\acidovorans_03_run.tsv',
 'MSFragger_Output\\aurantiaca_01_run.tsv',
 'MSFragger_Output\\aurantiaca_02_run.tsv',
 'MSFragger_Output\\aurantiaca_03_run.tsv',
 'MSFragger_Output\\baltica_01C_run.tsv',
 'MSFragger_Output\\baltica_01Inf_run.tsv',
 'MSFragger_Output\\baltica_02C_run.tsv',
 'MSFragger_Output\\baltica_02Inf_run.tsv',
 'MSFragger_Output\\baltica_03C_run.tsv',
 'MSFragger_Output\\baltica_03Inf_run.tsv',
 'MSFragger_Output\\cereus_01_run.tsv',
 'MSFragger_Output\\cereus_02_run.tsv',
 'MSFragger_Output\\cereus_03_run.tsv',
 'MSFragger_Output\\comes_Carb_run.tsv',
 'MSFragger_Output\\comes_Glc_run.tsv',
 'MSFragger_Output\\comes_LIB_run.tsv',
 'MSFragger_Output\\cryptum_01_run.tsv',
 'MSFragger_Output\\cryptum_02_run.tsv',
 'MSFragger_Output\\cryptum_03_run.tsv',
 'MSFragger_Output\\gilvus_01_run.tsv',
 'MSFragger_Output\\gilvus_02_run.tsv',
 'MSFragger_Output\\gilvus_03_r

In [5]:
def extract_scannum(Spectrum):
    pattern = r'\.\d*\.'
    lst = []
    for s in Spectrum:
        m = re.search(pattern, s)
        if m:
            m = m[0]
            m = m.replace('.', '')
            m = int(m)
            lst.append(m)
        else:
            lst.append(None)
    return (lst)

In [6]:
def combine_run_files(files):
    dfs = []
    for file in files:
        name = re.split('_run.tsv', file)
        name = name[0]
        pattern = r"\\"
        name = re.split(pattern, name)
        name = name[1]
        name = re.split('_', name)
        name = name[0]
        df = pd.read_csv(file, sep = '\t')
        keeps = ['scannum', 'best_locs', 'peptide']
        df = df[df.best_locs.notnull()]
        df = df[keeps]
        df['Organism'] = [name] * len(df)
        dfs.append(df)
    dfs = pd.concat(dfs)
    return (dfs)


In [7]:
def combine_psm_files(files):
    dfs = []
    i = 0
    for file in files:
        name = re.split('_psm.tsv', file)
        name = name[0]
        pattern = r"\\"
        name = re.split(pattern, name)
        name = name[1]
        keeps = ['scannum', 'Peptide', 'Observed Modifications', 'Protein ID', 'Gene', 'Spectrum']
        df = pd.read_csv(file, sep = '\t')   
        df = df[df['PeptideProphet Probability'] >=0.99]
        df = df[df['Observed Modifications'].notnull()]
        df = df[df.Gene.notnull()]
        df = df[~df.Gene.str.contains('Daci')]
        df['scannum'] = extract_scannum(df.Spectrum)
        df = df[keeps]
        df['Organism'] = [name] * len(df)        
        dfs.append(df)
    df = pd.concat(dfs)
    return(df)

In [8]:
combinded_psm_df = combine_psm_files(psm_files)
combined_run_df = combine_run_files(run_files)

C:\Users\nanel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
df = pd.merge(combinded_psm_df, combined_run_df)
df

,scannum,Peptide,Observed Modifications,Protein ID,Gene,Spectrum,Organism,best_locs,peptide
0,2637,AHALHGGMSQEQR,"Asn->Asp(0.984016), Deamidated(0.984016), Gln-...",E3FE31,deaD,Biodiversity_S_aurantiaca_CYE_aerobic_1_17July...,aurantiaca,qAEWIQATARK,QAEWIQATARK
1,2637,RPSSSHADDSPKPSGGASR,"Glu->Met(1.997892), Label:15N(2)(1.994070), La...",E3FWY3,STAUR_3242,Biodiversity_S_aurantiaca_CYE_aerobic_3_17July...,aurantiaca,qAEWIQATARK,QAEWIQATARK
2,2646,EISQTPTHHGHGQEGK,"15N-oxobutanoic(-18.023584), Dehydrated(-18.01...",Q095T6,thiC,Biodiversity_S_aurantiaca_CYE_aerobic_1_17July...,aurantiaca,eiSQTPTHHGHGQEGK,EISQTPTHHGHGQEGK
3,2659,VGAASKAPAKQASAK,"Asn->Gln(14.015650), Asp->Glu(14.015650), Gly-...",Q096H6,STAUR_0937,Biodiversity_S_aurantiaca_CYE_aerobic_1_17July...,aurantiaca,VGAASKAPAkQASAK,VGAASKAPAKQASAK
4,2659,VGAASKAPAKQASAK,"Asn->Gln(14.015650), Asp->Glu(14.015650), Gly-...",Q096H6,STAUR_0937,Biodiversity_S_aurantiaca_CYE_aerobic_1_17July...,aurantiaca,AAdVANDWMVR,AADVANDWMVR
...,...,...,...,...,...,...,...,...,...
238922,50567,LSNLQAMLPVLEAVVQTGKPLVIIAEDVEGEALATLVVNK,Val->Thr(1.979265),A0A176XGK2,groEL,Biodiversity_A_tumefaciens_R2A_aerobic_3_23Nov...,tumefaciens,SDLLpilyatatgtldkvqAEWRDDAALTVVLASK,SDLLPILYATATGTLDKVQAEWRDDAALTVVLASK
238923,50603,DVVEILAAEGFNIGR,"Ala->Gly(-14.015650), Gln->Asn(-14.015650), Gl...",A0A176WZL0,rplI,Biodiversity_A_tumefaciens_R2A_aerobic_3_23Nov...,tumefaciens,EVPLLANLFGTRQRIEWGLGLETGGLPALgQK,EVPLLANLFGTRQRIEWGLGLETGGLPALGQK
238924,50635,GFGFIQPDNGGTDVFVHISAVER,"Asn->Asp(0.984016), Deamidated(0.984016), Gln-...",A0A037XNX0,cspA,Biodiversity_A_tumefaciens_R2A_aerobic_3_23Nov...,tumefaciens,AIAGALTAIMTGVSYATSAEMAGELGPFPGFAPnrDNMLR,AIAGALTAIMTGVSYATSAEMAGELGPFPGFAPNRDNMLR
238925,50717,KLPVTSLLMALGMDGEDILSTFYTK,Asp->Met(16.013542),A0A176XCU3,rpoB,Biodiversity_A_tumefaciens_R2A_aerobic_3_23Nov...,tumefaciens,VLSVVGSVASEKAISMIAIHAHGIPHIFPQNVLAEADaAK,VLSVVGSVASEKAISMIAIHAHGIPHIFPQNVLAEADAAK


In [10]:
genes = df[['Gene', 'Organism']]
genes = genes.drop_duplicates()
genes = genes.groupby('Gene').count()
genes = pd.DataFrame(genes)
genes = genes[genes.Organism > 5]
genes = genes.index
homologous_genes = df[['Gene', 'Organism']]
homologous_genes = homologous_genes.drop_duplicates()
homologous_genes = homologous_genes[homologous_genes.Gene.isin(genes)]
homologous_genes.to_csv('Homologous_genes.csv')
len(genes)

254

Questions for Dr. Payne
1) Peptide vs peptide discrepancies
2) Missing scannums/ some in one but not the other